Teams Data Scraping

In [1]:
import requests
from bs4 import BeautifulSoup 
import json
import pandas as pd
import os

In [4]:
base_url = 'https://understat.com/league/EPL/'
season_names = ['2014', '2015', '2016', '2017', '2018','2019']

def season_scraper(season):
    url = base_url + season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    # data is under "scripts" tag. So, finding all the script tags
    script = soup.find_all('script')
    
    string_with_json_obj = ''

    for el in script:
        if 'teamsData' in el.text:
            string_with_json_obj = el.text.strip()
            
    #keeping only the part of string we need
    ind_start = string_with_json_obj.index("('") + 2
    ind_end = string_with_json_obj.index("')") 
    json_data = string_with_json_obj[ind_start: ind_end]

    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)#formatted json data